In [54]:
import pandas as pd
import datetime

In [55]:
sp500 = pd.read_csv('./stocks/^GSPC.csv')

In [56]:
sp500 = sp500.drop(["High", "Low", "Adj Close", "Volume"], axis=1)

In [57]:
sp500['Date']= pd.to_datetime(sp500['Date']) 

In [58]:
sp500.head(5)

,Date,Open,Close
0,2010-01-04,1116.560059,1132.989990
1,2010-01-05,1132.660034,1136.520020
2,2010-01-06,1135.709961,1137.140015
3,2010-01-07,1136.270020,1141.689941
4,2010-01-08,1140.520020,1144.979980


In [71]:
# Next day open, next day close
sp500[["1 Day Open", "1 Day Close"]] = sp500[["Open", "Close"]].shift(-1, axis=0)
# 2 day open, close
sp500[["2 Day Open", "2 Day Close"]] = sp500[["Open", "Close"]].shift(-2, axis=0)

In [73]:
# End of week close
end_of_week_list = list(map(lambda x: x + datetime.timedelta( (4-x.weekday()) % 7 ), sp500["Date"].tolist()))
# Sometimes the end of the week will be a holiday
for idx, x in enumerate(end_of_week_list):
    while sp500[(sp500["Date"] == str(x))].empty:
        x = x - datetime.timedelta(days=1)
    end_of_week_list[idx] = x
sp500["EOW Close"] = [sp500[(sp500["Date"] == str(x))]["Close"].tolist()[0] for x in end_of_week_list]

In [83]:
# Start of next week open
start_of_next_week_list = list(map(lambda x: x + datetime.timedelta(days=(7 - x.weekday())), sp500["Date"].tolist()))
# all(x[0] < x[1] for x in list(zip(end_of_week_list, start_of_week_list)))
# Sometimes the start of next week will be a holiday:
for idx, x in enumerate(start_of_next_week_list):
    while sp500[(sp500["Date"] == str(x))].empty and x < sp500.iloc[-1]["Date"]:
        x = x + datetime.timedelta(days=1)
    start_of_next_week_list[idx] = x
# Sorry this is a bit messy...
sp500["SOW Open"] = [sp500[(sp500["Date"] == str(x))]["Open"].tolist()[0] 
                     if not sp500[(sp500["Date"] == str(x))].empty else None for x in start_of_next_week_list]

In [85]:
# Next Week Open
sp500.head(10)

,Date,Open,Close,1 Day Open,1 Day Close,2 Day Open,2 Day Close,EOW Close,SOW Open
0,2010-01-04,1116.560059,1132.989990,1132.660034,1136.520020,1135.709961,1137.140015,1144.979980,1145.959961
1,2010-01-05,1132.660034,1136.520020,1135.709961,1137.140015,1136.270020,1141.689941,1144.979980,1145.959961
2,2010-01-06,1135.709961,1137.140015,1136.270020,1141.689941,1140.520020,1144.979980,1144.979980,1145.959961
3,2010-01-07,1136.270020,1141.689941,1140.520020,1144.979980,1145.959961,1146.979980,1144.979980,1145.959961
4,2010-01-08,1140.520020,1144.979980,1145.959961,1146.979980,1143.810059,1136.219971,1144.979980,1145.959961
5,2010-01-11,1145.959961,1146.979980,1143.810059,1136.219971,1137.310059,1145.680054,1136.030029,1136.030029
6,2010-01-12,1143.810059,1136.219971,1137.310059,1145.680054,1145.680054,1148.459961,1136.030029,1136.030029
7,2010-01-13,1137.310059,1145.680054,1145.680054,1148.459961,1147.719971,1136.030029,1136.030029,1136.030029
8,2010-01-14,1145.680054,1148.459961,1147.719971,1136.030029,1136.030029,1150.229980,1136.030029,1136.030029
9,2010-01-15,1147.719971,1136.030029,1136.030029,1150.229980,1147.949951,1138.040039,1136.030029,1136.030029


## Day Splitting
We should probably split each day into 2 when dealing with tweets, and generally try to predict next timestep, 2 timesteps out:
- If tweet happens after hours, predict next open
- If tweet happens during the day, predict close
- 2 day and eow/sow predictions shouldn't change

## Columns to drop:
- May as well drop high and low, as we don't know when during the day they occur
- Same thing with trading volume - DROP FOR NOW, MIGHT USE LATER

In [ ]:
sp500.to_csv("./stocks/spx_preprocessed.csv")